## Create index and add policies to Pinecone

In [1]:
import os

# Import the Pinecone library
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

index_name = "safety-policies"

/Users/lakshmi/Downloads/project1/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# only run this once
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [2]:
index = pc.Index(index_name)
model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Sat, 27 Dec 2025 15:18:33 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '38',
                                    'x-pinecone-request-id': '1901235862267172443',
                                    'x-pinecone-request-latency-ms': '37',
                                    'x-pinecone-response-duration-ms': '40'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'storageFullness': 0.0,
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [10]:
def tokenise_upsert(policies):
    to_upsert = []
    for p in policies:
        vector = model.encode(p['text']).tolist()
        to_upsert.append((p['id'], vector, {"description": p['text']}))

    index.upsert(vectors=to_upsert)

In [11]:
# Define your dynamic rules
policies = [
    {"id": "1", "text": "Medical Advice: Do not provide medical diagnoses or treatment suggestions."},
    {"id": "2", "text": "PII Protection: Never reveal or ask for social security numbers or home addresses."},
    {"id": "3", "text": "In life-threatening situations, do not provide DIY rescue steps. Instruct the user to contact local emergency services (e.g., 911) immediately."},
    {"id": "4", "text": "Do not provide legal interpretations, case law analysis, or specific legal strategies. Advise the user to consult a licensed attorney."},
    {"id": "5", "text": "Remain neutral on sensitive political or social issues. Do not take a stance that could be perceived as corporate lobbying or discrimination."}
]

# Generate embeddings and upsert
tokenise_upsert(policies)